# Data collection and cleaning

In this notebook, we focus on collecting the dataset necessary for creating our extended Fama-French model. We begin by gathering information regarding all listed stocks in the United States, which is facilitated using the Alpha Vantage API. The objective is to build a comprehensive dataset that incorporates various stock price characteristics and technical indicators, as well as the traditional Fama-French factors. The dataset includes the following variables:

1. Date (object): Represents the date of each observation, allowing us to align stock data chronologically for analysis.
2. Close (float64): The closing price of the stock for each day, which is essential for calculating returns and other price-based metrics.
3. RSI (Relative Strength Index, float64): A momentum indicator that measures the speed and change of recent price movements. RSI compares the magnitude of recent gains to recent losses to determine overbought or oversold conditions, typically using a range from 0 to 100. A value above 70 is generally considered overbought, while a value below 30 is oversold.
4. Volatility (float64): Represents the standard deviation of daily returns over a rolling window, providing an indication of how much the price of a stock fluctuates over time. High volatility indicates greater price swings, whereas low volatility suggests stability.
5. EPS (Earnings Per Share, float64): This represents the portion of a company’s profit attributed to each outstanding share of common stock, serving as a key measure of profitability.
6. Dividends (float64): The total amount of dividends paid out per share, which is used to evaluate the company's return to its shareholders.
7. Trading_Volume (int64): The number of shares traded over a specific period, which indicates investor interest and market liquidity for the stock.
8. Liquidity (float64): Defined as the product of trading volume and closing price, liquidity helps measure the ease with which an asset can be bought or sold in the market without affecting its price.
9. SMA_50 (Simple Moving Average, float64): The average of closing prices over a defined period, in this case, 50 days. SMA smooths out price data and is used to identify trends over a longer time frame.
10. MACD (Moving Average Convergence Divergence, float64): This indicator calculates the difference between two exponential moving averages (typically a 12-day EMA and a 26-day EMA) to identify momentum and possible trend reversals. A positive MACD value indicates a bullish trend, while a negative value suggests a bearish trend.
11. MACD_Signal (float64): The 9-day EMA of the MACD, which acts as a trigger for buy or sell signals based on crossovers with the MACD line.
12. MACD_Hist (float64): The difference between the MACD and its signal line. A positive histogram value indicates momentum in the bullish direction, while a negative value indicates bearish momentum.
13. ATR (Average True Range, float64): This indicator measures market volatility by analyzing the range between the high and low prices over a given period. A higher ATR value suggests increased volatility.
14. STOCH_K (Stochastic Oscillator %K, float64): This momentum indicator compares a particular closing price to a range of its prices over a specified period. The %K value provides insight into overbought or oversold conditions.
15. STOCH_D (Stochastic Oscillator %D, float64): The moving average of %K, which smoothens the stochastic value to provide signals of trend changes.
16. BB_Upper (Bollinger Bands Upper, float64): The upper band, calculated as a moving average of the closing prices plus two standard deviations. It represents a resistance level for the stock price.
17. BB_Middle (Bollinger Bands Middle, float64): The simple moving average (usually 20 days) that acts as the midpoint in the Bollinger Bands. It represents the baseline trend.
18. BB_Lower (Bollinger Bands Lower, float64): The lower band, calculated as a moving average of the closing prices minus two standard deviations. It represents a support level for the stock price.
19. OBV (On-Balance Volume, float64): A cumulative measure of trading volume that helps confirm trends. It adds the volume on up days and subtracts it on down days to gauge buying or selling pressure.
20. Ticker (object): The ticker symbol of the stock, which helps identify which company each row of data refers to.
21. Mkt-RF (float64): The market excess return, which is the return of the market portfolio minus the risk-free rate. It is a traditional Fama-French factor used to determine a stock’s exposure to market risk.
22. SMB (float64): The "Small Minus Big" factor, representing the additional return investors receive from investing in small-cap stocks over large-cap stocks. It is one of the core Fama-French factors to capture size risk.
23. HML (float64): The "High Minus Low" factor, which measures the additional return from investing in value stocks (high book-to-market ratio) over growth stocks (low book-to-market ratio). It captures value risk as per the Fama-French model.
24. RF (float64): The risk-free rate of return, which is usually the return on government treasury securities. It serves as a benchmark for assessing the excess return of an asset.
25. Stock_Return (float64): The daily return of the stock, calculated based on closing prices. It serves as the response variable in regression models to measure a stock’s performance.
26. Excess_Return (float64): The stock return minus the risk-free rate, which represents the return an investor receives above the risk-free rate.

After collecting the required stock data, we also gather Fama-French data to use in building our extended model. Our base model is the 5-factor Fama-French model, which includes:

1. Market excess return (Mkt-RF)
2. Small Minus Big (SMB)
3. High Minus Low (HML)
4. Risk-Free rate (RF)

After combining stock-specific characteristics with these traditional factors, we perform feature selection to determine the most influential variables. The goal is to extend the Fama-French model by incorporating additional technical indicators, enhancing the model's ability to explain stock returns beyond the traditional factors. By including momentum indicators, volatility measures, and trend analysis metrics, we aim to develop a more robust model that can better capture the complexities of stock price movements. This extended model provides a more comprehensive approach to evaluating and predicting stock returns, offering valuable insights for investors and analysts.

In [1]:
# Use pip, the Python package installer, to install the required libraries
pip install yfinance pandas pandas-ta numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 878.4 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=296b657c5737184b374c938030dfdd71b0425518ada812173040863405fbfe8d
  Stored in directory: /Users/meghagarwal/Library/Caches/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import statsmodels.api as sm

In [1]:
# Alpha Vantage API key
api_key = 'STQ2DO8BVL7FEP77'

# Alpha Vantage API endpoint for listing stock status
url = f'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={api_key}'

# Make the request to fetch stock listing data
response = requests.get(url)

# Save the data as CSV
with open('us_stocks.csv', 'w') as f:
    f.write(response.text)

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('us_stocks.csv')

# Filter for only active (listed) stocks
active_stocks = df[df['status'] == 'Active']

# Convert the active stock symbols into a list
listed_stocks = active_stocks['symbol'].tolist()

In [2]:
# Function to calculate technical indicators for each stock
def get_stock_data(ticker):
    try:
        # Fetch historical data for each ticker
        stock = yf.Ticker(ticker)
        df = stock.history(period="max")  # Get 1 year of historical data

        # Check if data is empty (stock might be delisted)
        if df.empty:
            print(f"No data for {ticker}, skipping...")
            return None

        # Calculate RSI (momentum-based factor)
        df['RSI'] = ta.rsi(df['Close'], length=14)

        # Calculate Volatility (standard deviation of returns)
        df['Daily_Return'] = df['Close'].pct_change()
        df['Volatility'] = df['Daily_Return'].rolling(window=20).std() * np.sqrt(252)

        # Get EPS and Dividends (we assume these are constant, as they are reported periodically)
        eps = stock.info.get('trailingEps', np.nan)  # EPS from Yahoo Finance
        dividends = stock.info.get('dividendRate', np.nan)  # Dividends

        # Add the same EPS and Dividends value for all dates
        df['EPS'] = eps
        df['Dividends'] = dividends

        # Trading volume and liquidity
        df['Trading_Volume'] = df['Volume']
        df['Liquidity'] = df['Trading_Volume'] * df['Close']  # Approximation of liquidity

        # Add the ticker to identify each stock
        df['Ticker'] = ticker
        
        # Add more technical indicators
        df['SMA_50'] = ta.sma(df['Close'], length=50)

        # MACD calculation
        macd_df = ta.macd(df['Close'], fast=12, slow=26, signal=9)
        df['MACD'] = macd_df['MACD_12_26_9']
        df['MACD_Signal'] = macd_df['MACDs_12_26_9']
        df['MACD_Hist'] = macd_df['MACDh_12_26_9']

        # ATR calculation
        df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)

        # Stochastic Oscillator calculation
        stoch_df = ta.stoch(df['High'], df['Low'], df['Close'])
        df['STOCH_K'] = stoch_df['STOCHk_14_3_3']
        df['STOCH_D'] = stoch_df['STOCHd_14_3_3']

        # Bollinger Bands calculation
        bb_df = ta.bbands(df['Close'], length=20, std=2)
        df['BB_Upper'] = bb_df['BBU_20_2.0']
        df['BB_Middle'] = bb_df['BBM_20_2.0']
        df['BB_Lower'] = bb_df['BBL_20_2.0']

        # On-Balance Volume (OBV) calculation
        df['OBV'] = ta.obv(df['Close'], df['Volume'])

        # Drop rows with missing data if necessary
        df.dropna(inplace=True)

        return df[['Close', 'RSI', 'Volatility', 'EPS', 'Dividends', 'Trading_Volume', 'Liquidity', 
                   'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Hist', 'ATR', 'STOCH_K', 'STOCH_D', 
                   'BB_Upper', 'BB_Middle', 'BB_Lower', 'OBV', 'Ticker']]

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Example list of stock tickers (replace with the actual list of US stocks)
tickers = listed_stocks # Add more tickers or your listed_stocks

# Fetch data for all tickers
all_stock_data = pd.DataFrame()

for ticker in tickers:
    stock_data = get_stock_data(ticker)
    if stock_data is not None:
        all_stock_data = pd.concat([all_stock_data, stock_data])

# Reset index for cleaner display
all_stock_data.reset_index(inplace=True)

$AACT-U: possibly delisted; No timezone found


No data for AACT-U, skipping...


$AACT-WS: possibly delisted; No timezone found


No data for AACT-WS, skipping...


AAGRW: Period 'max' is invalid, must be one of ['1d', '5d']


...

No data for ZXYZ-A, skipping...
No data for ZXZZT, skipping...


$ZYNE: possibly delisted; No timezone found


No data for ZYNE, skipping...


ZZK: Period 'max' is invalid, must be one of ['1d', '5d']


No data for ZZK, skipping...


In [4]:
# URL for the Fama-French data
ff_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"

# Read the Fama-French CSV data while skipping non-data rows
ff_data = pd.read_csv(ff_url, skiprows=3)

# Drop rows that contain non-numerical values in the 'Date' column
ff_data = ff_data[ff_data['Unnamed: 0'].str.isdigit()]  # Keep only rows where 'Unnamed: 0' contains numeric data

# Rename 'Unnamed: 0' to 'Date'
ff_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Convert the 'Date' column to datetime format
ff_data['Date'] = pd.to_datetime(ff_data['Date'], format='%Y%m%d')

# Set 'Date' as the index
ff_data.set_index('Date', inplace=True)

# Keep only the necessary columns (Mkt-RF, SMB, HML, RF)
ff_data = ff_data[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100  # Convert to decimal returns

In [5]:
# Ensure the stock data has a proper datetime index for merging
# Convert the 'Date' column to datetime format without time component
all_stock_data['Date'] = all_stock_data['Date'].dt.date  # Convert to just the date part (removes time)

# Set 'Date' as the index of the DataFrame to facilitate merging with Fama-French data
all_stock_data.set_index('Date', inplace=True)

# Merge the stock data with the Fama-French data using the datetime index
# The 'how' parameter is set to 'inner' to include only the dates that are present in both dataframes
merged_data = pd.merge(all_stock_data, ff_data, left_index=True, right_index=True, how='inner')

# Display the first few rows of the merged dataset to verify the merge was successful
merged_data.head()

,Close,RSI,Volatility,EPS,Dividends,Trading_Volume,Liquidity,SMA_50,MACD,MACD_Signal,...,STOCH_D,BB_Upper,BB_Middle,BB_Lower,OBV,Ticker,Mkt-RF,SMB,HML,RF
Date,,,,,,,,,,,,,,,,,,,,,
1962-03-13,1.531824,77.058560,0.187591,-3.07,0.40,36954,56607.028233,1.410197,0.034149,0.019461,...,94.563829,1.554945,1.425144,1.295342,289638.0,AA,0.0028,-0.003,0.0018,0.00009
1962-03-13,0.979397,64.137027,0.173134,5.00,3.52,5000,4896.987379,0.927940,0.012196,0.010148,...,87.966520,0.981313,0.956327,0.931342,24800.0,AEP,0.0028,-0.003,0.0018,0.00009
1962-03-13,0.149722,57.083646,0.129289,2.58,1.92,46464,6956.668928,0.144759,0.001146,0.001481,...,43.520463,0.150591,0.148766,0.146942,743424.0,BP,0.0028,-0.003,0.0018,0.00009
1962-03-13,0.133016,38.702509,0.186309,21.94,5.64,141600,18835.030460,0.137939,-0.001190,-0.000666,...,15.882031,0.140280,0.136298,0.132316,758400.0,CAT,0.0028,-0.003,0.0018,0.00009
1962-03-13,0.319304,57.587313,0.133359,1.62,0.80,21352,6817.784509,0.308230,0.003574,0.003357,...,81.403013,0.322844,0.315330,0.307815,75800.0,CNP,0.0028,-0.003,0.0018,0.00009


In [6]:
# Calculate daily stock returns
# 'pct_change()' computes the percentage change between the current and previous day
merged_data['Stock_Return'] = merged_data['Close'].pct_change()

# Calculate excess stock returns
# Excess return is the daily stock return minus the risk-free rate ('RF') from the Fama-French data
# This measures how much return a stock generates over the risk-free rate
merged_data['Excess_Return'] = merged_data['Stock_Return'] - merged_data['RF']

# Display the first few rows of the merged dataset to verify that the calculations are correct
merged_data.head()

,Close,RSI,Volatility,EPS,Dividends,Trading_Volume,Liquidity,SMA_50,MACD,MACD_Signal,...,BB_Middle,BB_Lower,OBV,Ticker,Mkt-RF,SMB,HML,RF,Stock_Return,Excess_Return
Date,,,,,,,,,,,,,,,,,,,,,
1962-03-13,1.531824,77.058560,0.187591,-3.07,0.40,36954,56607.028233,1.410197,0.034149,0.019461,...,1.425144,1.295342,289638.0,AA,0.0028,-0.003,0.0018,0.00009,NaN,NaN
1962-03-13,0.979397,64.137027,0.173134,5.00,3.52,5000,4896.987379,0.927940,0.012196,0.010148,...,0.956327,0.931342,24800.0,AEP,0.0028,-0.003,0.0018,0.00009,-0.360633,-0.360723
1962-03-13,0.149722,57.083646,0.129289,2.58,1.92,46464,6956.668928,0.144759,0.001146,0.001481,...,0.148766,0.146942,743424.0,BP,0.0028,-0.003,0.0018,0.00009,-0.847129,-0.847219
1962-03-13,0.133016,38.702509,0.186309,21.94,5.64,141600,18835.030460,0.137939,-0.001190,-0.000666,...,0.136298,0.132316,758400.0,CAT,0.0028,-0.003,0.0018,0.00009,-0.111580,-0.111670
1962-03-13,0.319304,57.587313,0.133359,1.62,0.80,21352,6817.784509,0.308230,0.003574,0.003357,...,0.315330,0.307815,75800.0,CNP,0.0028,-0.003,0.0018,0.00009,1.400500,1.400410


In [7]:
# Drop rows with any NaN values
clean_data = merged_data.dropna()

In [9]:
# Save the cleaned data to data.csv
clean_data.to_csv('data.csv')